<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🟢תהליך הסבת נקודות גבול החלקות לבנק"ל מודרני🟢

In [ ]:
print('13. Border Points:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as daskgpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
קריאת נתונים

In [ ]:
print('  13.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת נקודות גבול ברצף קדסטרי

In [ ]:
parcel_node_cols = ['PARCEL_SDE_', 'POINT_NAME', 'CONTROL_ID', 'REG_STATUS', 'POINT_MARK', 'geometry']

parcel_node = daskgpd.read_file(CNFG.Arcstorm + 'PARCEL_NODE.gdb', layer = 'PARCEL_NODE', columns = parcel_node_cols, npartitions = 16).compute()

del parcel_node_cols

print('         input 1/6 read')

<h1 style="text-align:right;font-size:100%">
שכבת נקודות גבול בתהליך

In [ ]:
in_process_cols = ['POINT_NAME', 'ACCURACY', 'MARK_CODE', 'TALAR_NUM', 'TALAR_YEAR', 'USERNAME', 'ENTITY_STATE', 'INSERTDATE', 'geometry']

in_process = daskgpd.read_file(CNFG.CADSDE + 'Points_InProcess.gdb', layer = 'Points_InProcess', columns = in_process_cols, npartitions = 16).compute()

del in_process_cols

print('         input 2/6 read')

<h1 style="text-align:right;font-size:100%">
שכבת נקודות גבול מבוטלות

In [ ]:
canceled_cols = ['POINT_NAME', 'ACCURACY', 'MARK_CODE', 'TALAR_NUM', 'TALAR_YEAR', 'USERNAME', 'INSERTDATE', 'geometry']

canceled = daskgpd.read_file(CNFG.Local + 'Points_InProcess_History_out_and_no_duplicates.gpkg', layer = 'Points_InProcess_History_out_and_no_duplicates', columns = canceled_cols, npartitions = 16).compute()

del canceled_cols

print('         input 3/6 read')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תהליכי קדסטר

In [ ]:
cpb_cols = ['CPBUniqueID', 'GlobalID', 'ProcessName', 'ProcessType', 'geometry']

cpb = daskgpd.read_file(CNFG.ProductPath, layer = 'CadasterProcessBorders', columns = cpb_cols, npartitions = 8).compute()

del cpb_cols

print('         input 4/6 read')

<h1 style="text-align:right;font-size:100%">
גבולות גושי קמ"ק

In [ ]:
cbc = cpb[cpb['ProcessType'] == 5]['geometry'].to_frame()
cbc['cbc'] = True

cpb.drop(columns = ['ProcessType', 'geometry'], inplace = True)


print('         input 5/6 read')

<h1 style="text-align:right;font-size:100%">
שכבת גושי ים תיכון

In [ ]:
medeteraninanBlocks = gpd.read_file(CNFG.Local + 'medeteraninanBlocks.gpkg', layer = 'medeteraninanBlocks', include_fields = ['geometry'])

medeteraninanBlocks['medeteraninan'] = True

print('         input 6/6 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים

<h1 style="text-align:right;font-size:125%">
:עיבוד נקודות גבול ברצף קדסטרי
<h1 style="text-align:right;font-size:100%">
הגדרת רשת קואורדינטות לרשת ישראל (2039)

In [ ]:
print('  13.b Cleaning data')

In [ ]:
parcel_node = parcel_node.to_crs(2039)

<h1 style="text-align:right;font-size:125%">
:עיבוד נקודות גבול בתהליך
<h1 style="text-align:right;font-size:100%">
סינון לנקודות יוצאות בלבד

In [ ]:
in_process = in_process[in_process['ENTITY_STATE'] == 1]
in_process.drop(columns = 'ENTITY_STATE', inplace = True)

<h1 style="text-align:right;font-size:100%">
הסרת נקודות כפולות

In [ ]:
in_process = in_process.sort_values(['TALAR_NUM','TALAR_YEAR', 'POINT_NAME', 'MARK_CODE', 'ACCURACY', 'INSERTDATE'])
in_process.drop_duplicates(keep = 'last', inplace = True)

<h1 style="text-align:right;font-size:125%">
איחוד נתוני נקודות מבוטלות עם נקודות בתהליך

In [ ]:
inprocess_data = pd.concat([canceled, in_process], ignore_index = True)\
                   .sort_values(['TALAR_NUM','TALAR_YEAR', 'POINT_NAME', 'MARK_CODE', 'ACCURACY', 'INSERTDATE'])

del [in_process, canceled]

<h1 style="text-align:right;font-size:100%">
הסרת כפילויות, רשומות עם תאריך מאוחר יותר ישארו

In [ ]:
inprocess_data.drop_duplicates(subset = 'geometry',
                               keep = 'last',
                               inplace = True)

inprocess_data.drop(columns = 'INSERTDATE', inplace = True)

<h1 style="text-align:right;font-size:100%">
מילוי ערכים חסרים בספרה 0

In [ ]:
inprocess_data[['MARK_CODE', 'ACCURACY', 'TALAR_NUM', 'TALAR_YEAR']] = inprocess_data[['MARK_CODE', 'ACCURACY', 'TALAR_NUM', 'TALAR_YEAR']].fillna(0)
inprocess_data[['MARK_CODE', 'ACCURACY', 'TALAR_NUM', 'TALAR_YEAR']] = inprocess_data[['MARK_CODE', 'ACCURACY', 'TALAR_NUM', 'TALAR_YEAR']].astype(int)

<h1 style="text-align:right;font-size:125%">
מיזוג עם שכבת גבולות תהליכי קדסטר מודרנית לקבלת מספר מזהה של התהליך שיצר/עידכן את הנקודה

In [ ]:
inprocess_data['ProcessName'] = inprocess_data['TALAR_NUM'].astype(str) + '/' + inprocess_data['TALAR_YEAR'].astype(str) 

inprocess_data = inprocess_data.merge(right = cpb, on = 'ProcessName', how = 'left')

inprocess_data.drop(columns = ['TALAR_NUM', 'TALAR_YEAR', 'ProcessName'], inplace = True)
del cpb

<h1 style="text-align:right;font-size:125%">
איחוד מרחבי עם של שכבת נקודות ברצף קדסטרי עם איחוד נקודות בתהליך

In [ ]:
print('  13.c Data conversion')

In [ ]:
parcel_node.rename(columns = {'POINT_NAME' : 'NodeName', 'PARCEL_SDE_' : 'PointUniqueID'}, inplace = True)
inprocess_data.rename(columns = {'POINT_NAME' : 'Name', 'GlobalID':'CreatedByRecord'}, inplace = True)

inprocess_data.geometry = inprocess_data.geometry.buffer(0.01)
BorderPoints = parcel_node.sjoin(inprocess_data, how = 'left', predicate = 'intersects')
BorderPoints.drop(columns = 'index_right', inplace = True)

del [parcel_node, inprocess_data]

<h1 style="text-align:right;font-size:100%">
הסרת נקודות כפולות בעלות אותה קואורדינטה

In [ ]:
BorderPoints = BorderPoints.drop_duplicates(subset = 'geometry', keep = 'first')

In [ ]:
print('  13.d Defining new Layer')

<h1 style="text-align:right;font-size:125%">
מיפוי ערכי סימון נקודת גבול
    <h1 style="text-align:right;font-size:100%">
הגדרת מילון מיפוי
        <h1 style="text-align:right;font-size:100%">
יצירת שדה חדש שלוקח לפי שדה שהגיע מנקודות בתהליך ואם חסר נלקח מנקודות ברצף (אם עדיין חסר יאוכלס בספרה 0)
            <h1 style="text-align:right;font-size:100%">
מיפוי הערכים לפי מילון והגדרת השדה כמספר שלם

In [ ]:
MARK_classifier = {0  : 0,
                   1  : 1,
                   2  : 2,
                   3  : 3,
                   4  : 4,
                   5  : 5,
                   6  : 6,
                   7  : 7,
                   8  : 8,
                   9  : 9,
                   10 : 10,
                   11 : 11,
                   12 : 12,
                   13 : 13,
                   14 : 14,
                   15 : 0,
                   20 : 20,
                   None : 0}

BorderPoints['MarkCode'] = BorderPoints['MARK_CODE'].fillna(BorderPoints['POINT_MARK'])
BorderPoints['MarkCode'] = BorderPoints['MarkCode'].map(MARK_classifier)
BorderPoints['MarkCode'] = BorderPoints['MarkCode'].fillna(0)
BorderPoints['MarkCode'] = BorderPoints['MarkCode'].astype(int)

BorderPoints.drop(columns = ['MARK_CODE', 'POINT_MARK'], inplace = True)
del MARK_classifier

<h1 style="text-align:right;font-size:125%">
התניית מקור נקודת גבול
<h1 style="text-align:right;font-size:100%">
הגדרת תנאי כאשר כל הנקודת בעלות סיווג קיים גדול מ-0 ותנאי תלוי בשם משתמש
<h1 style="text-align:right;font-size:100%">
יצירת שדות לשני התנאים שמתארים אם הערכים הרצויים מהתנאי
<h1 style="text-align:right;font-size:100%">
יצירת שדה אחד למקור הנקודה שעונה לשני התנאים

In [ ]:
BorderPoints['ACCURACY'] = BorderPoints['ACCURACY'].fillna(0)
CAD_Sourced_cond = ~BorderPoints['ACCURACY'].isin([0, None, np.nan])

SRV_Sourced_cond = BorderPoints['USERNAME'].isin(['shay', 'yaronl'])


BorderPoints['isCAD'] = np.where(CAD_Sourced_cond, 5, None)
BorderPoints['isSRV'] = np.where(SRV_Sourced_cond, 4, None)

BorderPoints['DataSource'] = BorderPoints['isCAD'].fillna(BorderPoints['isSRV'])
BorderPoints['DataSource'] = BorderPoints['DataSource'].fillna(3)
BorderPoints['DataSource'] = BorderPoints['DataSource'].astype(int)

BorderPoints.drop(columns = ['USERNAME', 'isCAD', 'isSRV'], inplace = True)
del CAD_Sourced_cond, SRV_Sourced_cond

<h1 style="text-align:right;font-size:125%">
התניית נקודת בקרה כנקודת גבול
<h1 style="text-align:right;font-size:100%">
הגדרת תנאי כאשר שדה של מזהה נקודות הבקרה הקיים גדול מ-0
<h1 style="text-align:right;font-size:100%">
יצירת שדה שעונה על התנאי בצורה בוליאנית

In [ ]:
BorderPoints['CONTROL_ID'] =  BorderPoints['CONTROL_ID'].fillna(0)
IsControlBorder_cond = BorderPoints['CONTROL_ID'] > 0

BorderPoints['IsControlBorder'] = np.where(IsControlBorder_cond, 1, 0)

BorderPoints.drop(columns = 'CONTROL_ID', inplace = True)
del IsControlBorder_cond

<h1 style="text-align:right;font-size:125%">
:התניית ומיפוי ערכי סיווג לנקודת הגבול
<h1 style="text-align:right;font-size:100%">
ACCURACY -ו REG_STATUS :מיפוי לפי שדות קיימים  

In [ ]:
ACCURACY_classifier = {0    : 24,
                       1    : 1,
                       2    : 13,
                       3    : 13,
                       4    : 13,
                       5    : 13,
                       6    : 24,
                       7    : 24,
                       8    : 24,
                       9    : 24,
                       12   : 12,
                       13   : 13,
                       24   : 24,
                       None : 24}

BorderPoints['ACCURACY'] = BorderPoints['ACCURACY'].map(ACCURACY_classifier)


REG_STATUS_classifier = {0    : 24,
                         1    : 24,
                         2    : 24,
                         3    : 24,
                         4    : 24,
                         5    : 24,
                         6    : 13,
                         7    : 24,
                         33   : 24,
                         None : 24}

BorderPoints['REG_STATUS'] = BorderPoints['REG_STATUS'].map(REG_STATUS_classifier)



BorderPoints['Class'] = BorderPoints['ACCURACY'].fillna(BorderPoints['REG_STATUS'])
BorderPoints['Class'] = BorderPoints['Class'].fillna(24)
BorderPoints['Class'] = BorderPoints['Class'].astype(int)


BorderPoints.drop(columns = ['ACCURACY', 'REG_STATUS'], inplace = True)
del [ACCURACY_classifier, REG_STATUS_classifier]

<h1 style="text-align:right;font-size:100%">
:מתן סיווג לפי מיקום גיאוגרפי
<h1 style="text-align:right;font-size:100%">
נקודות בתוך או על גבול גוש קמ"ק -
<h1 style="text-align:right;font-size:100%">
נקודות בתוך או על גבול גושי ים תיכון - 

In [ ]:
cbc.geometry = cbc.geometry.buffer(0.1)

BorderPoints = BorderPoints.sjoin(cbc, how = 'left', predicate = 'intersects')
BorderPoints['cbc'] = BorderPoints['cbc'].fillna(False)

cbc_class_cond = BorderPoints['cbc'] == True
cbc_class_value = 13

BorderPoints['Class'] = np.where(cbc_class_cond, cbc_class_value, BorderPoints['Class'])

BorderPoints.drop(columns =['index_right', 'cbc'], inplace = True)
del [cbc, cbc_class_cond, cbc_class_value]

In [ ]:
medeteraninanBlocks.geometry = medeteraninanBlocks.geometry.buffer(0.1)

BorderPoints = BorderPoints.sjoin(medeteraninanBlocks, how = 'left', predicate = 'intersects')
BorderPoints['medeteraninan'] = BorderPoints['medeteraninan'].fillna(False)

medeteraninan_class_cond = BorderPoints['medeteraninan'] == True
medeteraninan_class_value = 1

BorderPoints['Class'] = np.where(medeteraninan_class_cond, medeteraninan_class_value, BorderPoints['Class'])

BorderPoints.drop(columns =['index_right', 'medeteraninan'], inplace = True)
del [medeteraninanBlocks, medeteraninan_class_cond, medeteraninan_class_value]

<h1 style="text-align:right;font-size:100%">
הגדרת שדה שם נקודה לפי ערכים קיימים 

In [ ]:
BorderPoints['Name'] = BorderPoints['Name'].fillna(BorderPoints['NodeName'])
BorderPoints.drop(columns = 'NodeName', inplace = True)

<h1 style="text-align:right;font-size:100%">
הגדרת שדות של נקודות ציון לקואורדינטה

In [ ]:
BorderPoints['X'] = BorderPoints.geometry.x
BorderPoints['Y'] = BorderPoints.geometry.y

<h1 style="text-align:right;font-size:100%">
הסרת נקודות כפולות לפי קואורדינטה ושם נקודה

In [ ]:
BorderPoints = BorderPoints.drop_duplicates(['X', 'Y', 'Name'], keep = 'first')

<h1 style="text-align:right;font-size:100%">
השלמת מספרי זיהוי חסרים של תהליך יוצר

In [ ]:
BorderPoints['CPBUniqueID'] = BorderPoints['CPBUniqueID'].fillna(-2)
BorderPoints['CPBUniqueID'] = BorderPoints['CPBUniqueID'].astype('Int64')

BorderPoints['CreatedByRecord'] = BorderPoints['CreatedByRecord'].fillna(CNFG.FabricatedCreateProcessGUID)

<h1 style="text-align:right;font-size:100%">
הגדרת שדה מספר מזהה לנקודה מסוג מספר שלם 

In [ ]:
BorderPoints['PointUniqueID'] = BorderPoints['PointUniqueID'] + 5000000
BorderPoints['PointUniqueID'] = BorderPoints['PointUniqueID'].astype(int)

<h1 style="text-align:right;font-size:100%">
הסדרת נקודות נעולות לפי סיווג 1

In [ ]:
IsFixedCond = BorderPoints['Class'] == 1

BorderPoints['IsFixed'] = np.where(IsFixedCond, 1, 0)
BorderPoints['IsFixed'] = BorderPoints['IsFixed'].astype(int)

del IsFixedCond

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
שמירת תוצאות

In [ ]:
print('  13.e Exporting results to product folder')

<h1 style="text-align:right;font-size:100%">
בחירת נתיב
<h1 style="text-align:right;font-size:100%">
שמירה כטבלה
<h1 style="text-align:right;font-size:100%">
שמירה כשכבה גיאוגרפית

In [ ]:
json_dir = CNFG.JsonPath + 'BorderPoints.geojson'

BorderPoints.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([CNFG.BatchPath + 'BorderPoints-Save as Feature Class.bat']);